In [63]:
import pandas as pd
import numpy as np

df_train=pd.read_csv('/Users/chidam/Desktop/housing_prices_kaggle_train.csv')
df_test=pd.read_csv('/Users/chidam/Desktop/housing_price_kaggle_test_set.csv')
#Save the 'Id' column
train_ID = df_train['Id']
test_ID = df_test['Id']
# df_train.drop("Id", axis = 1, inplace = True)
# df_test.drop("Id", axis = 1, inplace = True)
df_train = df_train.drop(df_train[(df_train['GrLivArea']>4000) & (df_train['SalePrice']<300000)].index)
ntrain = df_train.shape[0]
ntest = df_test.shape[0]
df_train['SalePrice']=np.log1p(df_train['SalePrice'])
y_train = df_train.SalePrice.values
df_train.drop(['SalePrice'], axis=1, inplace=True)
entire_data = pd.concat((df_train, df_test)).reset_index(drop=True)


In [64]:
entire_data.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2008,WD,Normal
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,5,2007,WD,Normal
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,9,2008,WD,Normal
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,0,NaN,NaN,NaN,0,12,2008,WD,Normal


In [3]:
entire_data.shape

(2917, 79)

In [65]:
#Fill in the missing values

entire_data["PoolQC"] = entire_data["PoolQC"].fillna("None")
entire_data["MiscFeature"] = entire_data["MiscFeature"].fillna("None")
entire_data["Alley"] = entire_data["Alley"].fillna("None")
entire_data["Fence"] = entire_data["Fence"].fillna("None")
entire_data["FireplaceQu"] = entire_data["FireplaceQu"].fillna("None")
entire_data["LotFrontage"] = entire_data.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))
                                                                                         
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    entire_data[col] = entire_data[col].fillna('None')

for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    entire_data[col] = entire_data[col].fillna(0)
    
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    entire_data[col] = entire_data[col].fillna(0)
    
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    entire_data[col] = entire_data[col].fillna('None')
    
entire_data["MasVnrType"] = entire_data["MasVnrType"].fillna("None")
entire_data["MasVnrArea"] = entire_data["MasVnrArea"].fillna(0)

entire_data['MSZoning'] = entire_data['MSZoning'].fillna(entire_data['MSZoning'].mode()[0])

entire_data = entire_data.drop(['Utilities'], axis=1)

entire_data["Functional"] = entire_data["Functional"].fillna("Typ")

entire_data['Electrical'] = entire_data['Electrical'].fillna(entire_data['Electrical'].mode()[0])

entire_data['KitchenQual'] = entire_data['KitchenQual'].fillna(entire_data['KitchenQual'].mode()[0])

entire_data['Exterior1st'] = entire_data['Exterior1st'].fillna(entire_data['Exterior1st'].mode()[0])
entire_data['Exterior2nd'] = entire_data['Exterior2nd'].fillna(entire_data['Exterior2nd'].mode()[0])

entire_data['SaleType'] = entire_data['SaleType'].fillna(entire_data['SaleType'].mode()[0])

entire_data['MSSubClass'] = entire_data['MSSubClass'].fillna("None")




In [66]:
import calendar
entire_data['MoSold']=entire_data['MoSold'].apply(lambda x: calendar.month_abbr[x])

entire_data.loc[(entire_data['MoSold']=='Jun') | (entire_data['MoSold']=='Jul') | (entire_data['MoSold']=='Aug'),'Season']='Summer'
entire_data.loc[(entire_data['MoSold']=='Mar') | (entire_data['MoSold']=='Apr') | (entire_data['MoSold']=='May'),'Season']= 'Spring'
entire_data.loc[(entire_data['MoSold']=='Sep') | (entire_data['MoSold']=='Oct') | (entire_data['MoSold']=='Nov'),'Season']='Autumn'
entire_data.loc[(entire_data['MoSold']=='Jan') | (entire_data['MoSold']=='Feb') | (entire_data['MoSold']=='Dec'),'Season']='Winter'



In [67]:
entire_data['MSSubClass'] = entire_data['MSSubClass'].apply(str)
entire_data['Season']=entire_data['Season'].astype(str)

#Changing OverallCond into a categorical variable
entire_data['OverallCond'] = entire_data['OverallCond'].astype(str)


#Year and month sold are transformed into categorical features.
entire_data['YrSold'] = entire_data['YrSold'].astype(str)
entire_data['MoSold'] = entire_data['MoSold'].astype(str)

In [68]:
from sklearn.preprocessing import LabelEncoder
cols = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir', 'MSSubClass', 'OverallCond', 
        'YrSold', 'MoSold', 'Season')
# process columns, apply LabelEncoder to categorical features
for c in cols:
    lbl = LabelEncoder() 
    lbl.fit(list(entire_data[c].values)) 
    entire_data[c] = lbl.transform(list(entire_data[c].values))

In [69]:
entire_data['TotalSF'] = entire_data['TotalBsmtSF'] + entire_data['1stFlrSF'] + entire_data['2ndFlrSF']


In [70]:
entire_data.drop("Id", axis = 1, inplace = True)
# df_test.drop("Id", axis = 1, inplace = True)

In [71]:
entire_data.columns[entire_data.isnull().any()]

Index([], dtype='object')

In [72]:
from scipy.stats import norm, skew
numeric_feats = entire_data.dtypes[entire_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = entire_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
# print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})

In [73]:
skewness = skewness[abs(skewness) > 0.75]
print("There are {} skewed numerical features to Box Cox transform".format(skewness.shape[0]))

from scipy.special import boxcox1p
skewed_features = skewness.index
lam = 0.15
for feat in skewed_features:
    #all_data[feat] += 1
    entire_data[feat] = boxcox1p(entire_data[feat], lam)

There are 60 skewed numerical features to Box Cox transform


In [74]:
entire_data = pd.get_dummies(entire_data)

In [75]:
train_set= entire_data[:ntrain]
test_set = entire_data[ntrain:]

In [76]:
X_train=train_set.values
X_test=test_set.values

In [77]:
from sklearn.linear_model import ElasticNet, Lasso,  BayesianRidge, LassoLarsIC
from sklearn.ensemble import RandomForestRegressor,  GradientBoostingRegressor
from sklearn.kernel_ridge import KernelRidge
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import RobustScaler
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone
from sklearn.model_selection import KFold, cross_val_score, train_test_split
from sklearn.metrics import mean_squared_error
import xgboost as xgb
import lightgbm as lgb

In [20]:
# from bayes_opt import BayesianOptimization


In [78]:
lasso = make_pipeline(RobustScaler(), Lasso(alpha =0.0005, random_state=1))
ENet = make_pipeline(RobustScaler(), ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3))
KRR = KernelRidge(alpha=0.6, kernel='polynomial', degree=2, coef0=2.5)
GBoost = GradientBoostingRegressor(n_estimators=3000, learning_rate=0.05,
                                   max_depth=4, max_features='sqrt',
                                   min_samples_leaf=15, min_samples_split=10, 
                                   loss='huber', random_state =5)

xgb = xgb.XGBRegressor(colsample_bytree=0.4603, gamma=0.0468, 
                             learning_rate=0.05, max_depth=3, 
                             min_child_weight=1.7817, n_estimators=2200,
                             reg_alpha=0.4640, reg_lambda=0.8571,
                             subsample=0.5213, silent=1,
                             random_state =7, nthread = -1)
lgb = lgb.LGBMRegressor(objective='regression',num_leaves=5,
                              learning_rate=0.05, n_estimators=720,
                              max_bin = 55, bagging_fraction = 0.8,
                              bagging_freq = 5, feature_fraction = 0.2319,
                              feature_fraction_seed=9, bagging_seed=9,
                              min_data_in_leaf =6, min_sum_hessian_in_leaf = 11)


In [79]:

class stack_average_models(BaseEstimator, RegressorMixin, TransformerMixin):
    
    def __init__(self, basemodels, metamodel, n_folds=5):
        self.basemodels=basemodels
        self.metamodel=metamodel
        self.n_folds=n_folds
#         print(self.basemodels)
        
    def fit(self, X, y):
        import numpy as np
        self.basemodels_ = [list() for x in self.basemodels]
        self.metamodel_ = clone(self.metamodel)
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=156)
        
        holdout_predictions = np.zeros((X.shape[0], len(self.basemodels)))
        for ind, model in enumerate(self.basemodels):
            for train_ind, holdout_ind in kfold.split(X, y):
                instance = clone(model)
                self.basemodels_[ind].append(instance)
                instance.fit(X[train_ind], y[train_ind])
                y_pred = instance.predict(X[holdout_ind])
                holdout_predictions[holdout_ind, ind] = y_pred
                
         
        self.metamodel_.fit(holdout_predictions, y)
        return self
   
    #Do the predictions of all base models on the test data and use the averaged predictions as 
    #meta-features for the final prediction which is done by the meta-model
    def predict(self, X):
        import numpy as np
        meta_features = np.column_stack([np.column_stack([model.predict(X) for model in basemodels]).mean(axis=1) for basemodels in self.basemodels_ ])
        return self.metamodel_.predict(meta_features)
        

In [80]:
stacked_meta_lasso = stack_average_models(basemodels = (ENet, GBoost, KRR),
                                                metamodel = lasso)

In [81]:
stacked_meta_lasso.fit(X_train, y_train)
stacked_meta_pred = np.expm1(stacked_meta_lasso.predict(X_test))

In [82]:
xgb.fit(X_train, y_train)
xgb_pred = np.expm1(xgb.predict(X_test))

In [83]:
lgb.fit(X_train, y_train)
lgb_pred = np.expm1(lgb.predict(X_test))

In [88]:
ensemble = stacked_meta_pred*0.90 +xgb_pred*0.05 + lgb_pred *0.05

In [89]:


df_submissionfeb27_trial= pd.DataFrame({"ID" : test_ID, "SalePrice" : ensemble})
df_submissionfeb27_trial.to_csv("/Users/chidam/Desktop/Ames_feb27_trial.csv", index=False)




In [90]:
stacked_meta_Enet = stack_average_models(basemodels = (GBoost, xgb, lgb),
                                                 metamodel = ENet)



In [91]:
stacked_meta_Enet.fit(X_train, y_train)
stacked_pred_enet = np.expm1(stacked_meta_Enet.predict(X_test))

In [92]:
lasso.fit(X_train, y_train)
# krr_train_pred = Krr.predict(X_train)
lasso_pred = np.expm1(lasso.predict(X_test))

In [93]:
KRR.fit(X_train, y_train)
# krr_train_pred = Krr.predict(X_train)
KRR_pred = np.expm1(KRR.predict(X_test))

In [95]:
ensemble2 = stacked_pred_enet*0.77 + KRR_pred*0.07 + lasso_pred*0.14

In [96]:
#Did not get a chance to submit the following result
df_submissionfeb27_trial2= pd.DataFrame({"ID" : test_ID, "SalePrice" : ensemble2})
df_submissionfeb27_trial2.to_csv("/Users/chidam/Desktop/Ames_feb27_trial2.csv", index=False)



